In [ ]:
# -------------------- Mount Google Drive --------------------
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Cukup pasang rasterio (dan shapely bila diperlukan). Jangan menyentuh numpy/pandas/scipy/ sklearn bawaan Colab.
!pip install -q "rasterio==1.4.1" "shapely==2.0.6"

# **BAU Scenario**

In [ ]:
# ============================================================
# BAU 2032 — CA–Markov (single script, Colab-ready)
# ============================================================
# - Installs dependencies (with ABI-safe pinned versions) and mounts Google Drive
# - Loads PL_2016, PL_2024, and SUIT_2016_all.npy
# - Aggregates suitability → 3 channels (Sawah, Terbangun, Lainnya)
# - Builds Markov transition matrix T(2016→2024)
# - Runs quantity-constrained CA from 2024 → 2032
# - Saves raster output, CSV area summary, and subplot visualization
# ============================================================

# -------------------- Bootstrap: dependencies & mount --------------------
import sys, os

def _maybe_install_and_restart():
    try:
        import rasterio  # noqa
        import numpy as np  # noqa
        return
    except Exception as e:
        print("[Setup] Installing pinned versions for compatibility...", e)
        # Install versions compatible with Colab
        # Note: This will restart the runtime after installation
        os.system(
            "pip -q install --upgrade --force-reinstall --no-cache-dir "
            " \"numpy==1.26.4\" \"rasterio==1.3.9\" "
            " \"scipy==1.11.4\" \"pandas==2.1.4\" \"matplotlib==3.8.2\" "
            " \"scikit-learn==1.3.2\" \"shapely==2.0.1\""
        )
        print("[Setup] Restarting runtime to finalize installations...")
        os.kill(os.getpid(), 9)

_maybe_install_and_restart()

# Mount Google Drive (idempotent)
try:
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive', force_remount=True)
except Exception as _e:
    print("[Info] Not running on Colab or Drive mount failed. Proceeding without mount.")

# -------------------- Imports (after setup) --------------------
import numpy as np
import rasterio
import pandas as pd
from scipy.ndimage import uniform_filter
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

# -------------------- Paths --------------------
SAWAH_DIR = "/content/drive/MyDrive/Sawah2"    # expects PL_2016.tif, PL_2024.tif, SUIT_2016_all.npy
CHGDIR    = "/content/drive/MyDrive/Land Use/Change_Analysis"
REP_DIR   = os.path.join(CHGDIR, "reports")
os.makedirs(CHGDIR, exist_ok=True); os.makedirs(REP_DIR, exist_ok=True)

# -------------------- Utility functions --------------------

def read_raster(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {path}")
    with rasterio.open(path) as src:
        return src.read(1), src.profile


def save_with_colormap(arr, profile, path, nodata=255):
    prof = profile.copy()
    prof.update(dtype=rasterio.uint8, nodata=nodata, count=1, compress="lzw",
                height=arr.shape[0], width=arr.shape[1])
    with rasterio.open(path, 'w', **prof) as dst:
        dst.write(arr.astype(np.uint8), 1)
        dst.write_colormap(1, {
            1:(46,139,87,255),   # Sawah
            2:(220,20,60,255),   # Terbangun
            3:(30,144,255,255),  # Lainnya
            255:(0,0,0,0)
        })
    print("✅ Saved:", path)


def ha_per_pixel(profile):
    t = profile['transform']
    return abs(t.a)*abs(t.e)/10_000.0


def area_by_class(arr, ha_pix):
    return {c: float((arr==c).sum())*ha_pix for c in (1,2,3)}

# -------------------- Load maps & suitability --------------------
PL_16, _       = read_raster(os.path.join(SAWAH_DIR, "PL_2016.tif"))
PL_24, PROFILE = read_raster(os.path.join(SAWAH_DIR, "PL_2024.tif"))
H, W = PL_24.shape

SUIT_ALL_PATH     = os.path.join(SAWAH_DIR, "SUIT_2016_all.npy")
SUIT_CLASSES_PATH = os.path.join(SAWAH_DIR, "SUIT_2016_all_classes.npy")  # optional
if not os.path.exists(SUIT_ALL_PATH):
    raise FileNotFoundError(f"Suitability file not found: {SUIT_ALL_PATH}")

suits = np.load(SUIT_ALL_PATH)
if suits.ndim != 3 or suits.shape[1:] != (H,W):
    raise ValueError(f"Invalid SUIT_2016_all.npy dimensions: {suits.shape}; expected (C,{H},{W})")

if os.path.exists(SUIT_CLASSES_PATH):
    suit_classes = np.load(SUIT_CLASSES_PATH)
else:
    suit_classes = np.arange(suits.shape[0])

# Normalize per-pixel probabilities to sum = 1
suits = np.nan_to_num(suits.astype(np.float32), nan=0.0, posinf=0.0, neginf=0.0)
ss = suits.sum(axis=0, keepdims=True); ss[ss==0]=1
suits = suits/ss

# Aggregate to 3 channels: Sawah(1), Terbangun(2), Lainnya(rest)
GROUPS = {"Sawah":[1], "Lahan_Terbangun":[2], "Lainnya":"rest"}
cls_list = list(suit_classes.tolist())
stacks, names, used = [], [], set()
for gname, members in GROUPS.items():
    if members == "rest":
        idx = [i for i in range(suits.shape[0]) if i not in used]
    else:
        idx = [cls_list.index(m) for m in members if m in cls_list]
    if len(idx) == 0:
        stacks.append(np.zeros((H,W), np.float32))
    else:
        used.update(idx)
        stacks.append(np.nansum(suits[idx,...], axis=0).astype(np.float32))
    names.append(gname)
SUIT_3 = np.stack(stacks, axis=0).astype(np.float32)
ss3 = SUIT_3.sum(axis=0, keepdims=True); ss3[ss3==0]=1
SUIT_3 = SUIT_3/ss3
print("Aggregated channels:", names)

# -------------------- Markov transition matrix T(2016→2024) --------------------

def transition_matrix(start, end, num_classes=3, nodata=255):
    M = np.zeros((num_classes,num_classes), np.float64)
    m = (start!=nodata) & (end!=nodata)
    s = start[m]; e = end[m]
    for i in range(1, num_classes+1):
        sel = (s==i)
        if sel.any():
            for j in range(1, num_classes+1):
                M[i-1, j-1] = np.sum(e[sel]==j)
    row = M.sum(axis=1, keepdims=True); row[row==0]=1
    return M/row

T_16_24 = transition_matrix(PL_16, PL_24)
print("\n=== MARKOV TRANSITION 2016→2024 ===\n", np.round(T_16_24, 4))

# -------------------- Quantity-constrained CA (2024→2032) --------------------

def neighborhood_fraction(label_map, cls, size=5):
    return uniform_filter((label_map==cls).astype(np.float32), size=size, mode='nearest')


def compute_scores(current_map, T, suit3, neigh_weight=0.25, neigh_size=5, nodata=255):
    H,W = current_map.shape
    valid = (current_map!=nodata)
    ii, jj = np.where(valid)

    n1 = neighborhood_fraction(current_map, 1, neigh_size)
    n2 = neighborhood_fraction(current_map, 2, neigh_size)
    n3 = neighborhood_fraction(current_map, 3, neigh_size)
    LP = (1.0 - neigh_weight)*suit3 + neigh_weight*np.stack([n1,n2,n3], axis=0)

    cur = current_map[valid]
    S = np.zeros((3, ii.size), dtype=np.float32)
    for k in (1,2,3):
        S[k-1,:] = T[cur-1, k-1] * LP[k-1, ii, jj]

    order  = np.argsort(-S, axis=0)
    best   = order[0,:] + 1
    second = order[1,:] + 1
    margin = S[order[0,:], np.arange(S.shape[1])] - S[order[1,:], np.arange(S.shape[1])]
    return S, valid, (best, second, margin)


def targets_from_markov(initial_map, T, nodata=255):
    h0 = np.array([(initial_map==1).sum(), (initial_map==2).sum(), (initial_map==3).sum()], float)
    tgt_f = h0 @ T
    tgt = np.floor(tgt_f).astype(int)
    gap = int(h0.sum() - tgt.sum())
    if gap!=0:
        frac = tgt_f - np.floor(tgt_f); order = np.argsort(-frac)
        for i in order[:abs(gap)]: tgt[i] += 1 if gap>0 else -1
    return tgt


def allocate_with_targets(S, valid_mask, target_counts, info, nodata=255, max_iter=5):
    H,W = valid_mask.shape
    ii, jj = np.where(valid_mask)
    best, second, margin = info
    alloc = best.copy()

    def counts(v): return np.array([(v==1).sum(), (v==2).sum(), (v==3).sum()], np.int64)

    def reassign(from_cls, to_cls, need):
        if need<=0: return 0
        idx_from = np.where(alloc==from_cls)[0]
        if idx_from.size==0: return 0
        idx2 = idx_from[ second[idx_from]==to_cls ]
        if idx2.size==0: return 0
        take = min(need, idx2.size)
        sel = idx2[np.argsort(margin[idx2])[:take]]
        alloc[sel] = to_cls
        return take

    for _ in range(max_iter):
        diff = target_counts - counts(alloc)
        if np.all(diff==0): break
        for s_idx in np.argsort(diff):
            if diff[s_idx] >= 0: continue
            from_cls = s_idx+1; need_rm = -diff[s_idx]
            for d_idx in np.where(diff>0)[0]:
                to_cls = d_idx+1
                moved = reassign(from_cls, to_cls, min(need_rm, diff[d_idx]))
                need_rm -= moved; diff[d_idx] -= moved
                if need_rm<=0: break

    out = np.full((H,W), nodata, np.uint8)
    out[ii, jj] = alloc.astype(np.uint8)
    return out


def quantity_constrained_ca(initial_map, T, suit3,
                            neigh_weight=0.25, neigh_size=5, qc_iters=3, nodata=255):
    target = targets_from_markov(initial_map, T, nodata)
    cur = initial_map.copy()
    for _ in range(qc_iters):
        S, valid, info = compute_scores(cur, T, suit3,
                                        neigh_weight=neigh_weight, neigh_size=neigh_size, nodata=nodata)
        cur = allocate_with_targets(S, valid, target, info, nodata=nodata, max_iter=5)
    return cur, target

# Run CA from 2024 → 2032 using T(16→24)
pred_2032, target_2032 = quantity_constrained_ca(
    initial_map=PL_24, T=T_16_24, suit3=SUIT_3,
    neigh_weight=0.25, neigh_size=5, qc_iters=3, nodata=255
)

# -------------------- Save outputs --------------------
out_tif = os.path.join(CHGDIR, "Prediksi_2032_BAU_CA_Markov_from2016to2024.tif")
save_with_colormap(pred_2032, PROFILE, out_tif)

ha_pix = ha_per_pixel(PROFILE)
luas24 = area_by_class(PL_24, ha_pix)
luas32 = area_by_class(pred_2032, ha_pix)

summary = pd.DataFrame([
    ["Sawah",     luas24[1], luas32[1], luas32[1]-luas24[1]],
    ["Terbangun", luas24[2], luas32[2], luas32[2]-luas24[2]],
    ["Lainnya",   luas24[3], luas32[3], luas32[3]-luas24[3]],
], columns=["Class","2024 (ha)","2032 BAU (ha)","Δ(ha)"])

csv_out = os.path.join(REP_DIR, "area_summary_BAU_2032_from2016to2024.csv")
summary.to_csv(csv_out, index=False)
print("\n=== AREA SUMMARY ===\n", summary.round(2))
print("✅ Saved:", csv_out)

# -------------------- Visualization --------------------

def show_classmap(arr, title, nodata=255):
    labels = ['Sawah (1)', 'Terbangun (2)', 'Lainnya (3)']
    cmap = ListedColormap(['#2e8b57', '#dc143c', '#1e90ff'])
    m = np.ma.masked_where(arr == nodata, arr)
    plt.imshow(m, cmap=cmap, vmin=1, vmax=3)
    plt.title(title, fontweight='bold')
    plt.axis('off')
    patches = [mpatches.Patch(color=cmap(i), label=labels[i]) for i in range(3)]
    return patches

plt.figure(figsize=(12,6), dpi=600)
plt.subplot(1,2,2)
legend_handles = show_classmap(pred_2032, "BAU Scenario")

plt.legend(handles=legend_handles,
           loc='lower center',
           bbox_to_anchor=(0.5, -0.05),
           ncol=3,
           frameon=False,
           prop={'weight': 'bold'})

plt.tight_layout()
plt.show()

print("✅ Completed.")


# **ZRFP Scenario**

In [ ]:
# ============================================================
# PROJECTION 2032 — Probabilistic CA + Ske_B4B5 (super-protect rice fields; no Markov)
# ============================================================
# - WITHOUT transition matrix (no T / T²)
# - Protection: max 10% of rice fields within B4B5 allowed to convert (adaptive by pressure)
# - Endogenous (no quota): E = r_global(2016→2024) × POLICY_INTENSITY
# - Distribution: Random Forest suitability (BU) + neighborhood of built-up cells
# - BU absorbing; only 1/3 → 2 transitions allowed (1→3 blocked)
# - Aggressive paddy-loss reduction: weak source-share, protection & soft caps, edge softener
# ============================================================

# -------------------- Bootstrap: dependencies & mount --------------------
import sys, os
def _maybe_install_and_restart():
    try:
        import rasterio  # noqa
        import numpy as np  # noqa
        return
    except Exception as e:
        print("[Setup] Installing pinned versions ...", e)
        os.system(
            "pip -q install --upgrade --force-reinstall --no-cache-dir "
            " \"numpy==1.26.4\" \"rasterio==1.3.9\" "
            " \"scipy==1.11.4\" \"pandas==2.1.4\" \"matplotlib==3.8.2\" "
            " \"scikit-learn==1.3.2\" \"shapely==2.0.1\""
        )
        os.kill(os.getpid(), 9)
_maybe_install_and_restart()

try:
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive', force_remount=True)
except Exception:
    print("[Info] Drive mount skipped.")

# -------------------- Imports --------------------
import numpy as np, rasterio, pandas as pd
from scipy.ndimage import uniform_filter
import matplotlib.pyplot as plt, matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

# -------------------- Paths --------------------
SAWAH_DIR = "/content/drive/MyDrive/Sawah2"    # PL_2016.tif, PL_2024.tif, SUIT_2016_all.npy, Ske_B4B5.tif
CHGDIR    = "/content/drive/MyDrive/Land Use/Change_Analysis"
REP_DIR   = os.path.join(CHGDIR, "reports")
os.makedirs(CHGDIR, exist_ok=True); os.makedirs(REP_DIR, exist_ok=True)

# -------------------- Constants & Settings --------------------
NODATA       = 255
NEIGH_WEIGHT = 0.40
NEIGH_SIZE   = 9
QC_ITERS     = 3          # ↓ fewer iterations to reduce repeated transitions
RAND_SEED    = 42
np.random.seed(RAND_SEED)

# ======= Policy settings (strong anti-rice-field conversion, non-quota) =======
POLICY_INTENSITY = 1.6     # ↓ reduces overall Σp (still endogenous)
# Source weighting: weaken sawah, strengthen “others” (shift 3→2)
W1 = 0.30                  # ↓ weaker drive 1→2
W3 = 1.70                  # ↑ stronger drive 3→2

# Rice field protection in B4B5 zones
PROTECT_B4B5_FACTOR = 0.20 # extra penalty for 1→2 transitions inside B4B5
CAP_P_SAWAH  = 0.020       # global cap for 1→2 probability (2% per iteration)
CAP_P_B4B5   = 0.005       # extra-tight cap within B4B5 (0.5% per iteration)

# Source sharing (non-quota; proportional pressure)
USE_SOURCE_SHARE = True
E1_SHARE = 0.15            # only 15% of expected expansion sourced from rice fields

# -------------------- Utility functions --------------------
def read_raster(path):
    if not os.path.exists(path): raise FileNotFoundError(f"File not found: {path}")
    with rasterio.open(path) as src: return src.read(1), src.profile

def save_with_colormap(arr, profile, path, nodata=255):
    prof = profile.copy()
    prof.update(dtype=rasterio.uint8, nodata=nodata, count=1, compress="lzw",
                height=arr.shape[0], width=arr.shape[1])
    with rasterio.open(path, 'w', **prof) as dst:
        dst.write(arr.astype(np.uint8), 1)
        dst.write_colormap(1, {1:(46,139,87,255), 2:(220,20,60,255), 3:(30,144,255,255), 255:(0,0,0,0)})
    print("✅ Saved:", path)

def ha_per_pixel(profile):
    t = profile['transform']; return abs(t.a)*abs(t.e)/10_000.0

def area_by_class(arr, ha_pix):
    return {c: float((arr==c).sum())*ha_pix for c in (1,2,3)}

def neighborhood_fraction(label_map, cls, size=5):
    return uniform_filter((label_map==cls).astype(np.float32), size=size, mode='nearest')

# -------------------- Load data & suitability --------------------
PL_16, _       = read_raster(os.path.join(SAWAH_DIR, "PL_2016.tif"))
PL_24, PROFILE = read_raster(os.path.join(SAWAH_DIR, "PL_2024.tif"))
H, W = PL_24.shape

suits = np.load(os.path.join(SAWAH_DIR, "SUIT_2016_all.npy")).astype(np.float32)
if suits.ndim != 3 or suits.shape[1:] != (H,W): raise ValueError(f"SUIT_2016_all.npy shape {suits.shape} ≠ (C,{H},{W})")
classes_path = os.path.join(SAWAH_DIR, "SUIT_2016_all_classes.npy")
suit_classes = np.load(classes_path) if os.path.exists(classes_path) else np.arange(suits.shape[0])

suits = np.nan_to_num(suits, nan=0.0, posinf=0.0, neginf=0.0)
ss = suits.sum(axis=0, keepdims=True); ss[ss==0]=1; suits /= ss

# Aggregate into 3 channels
cls_list = list(suit_classes.tolist()); stacks, used = [], set()
GROUPS = {"Sawah":[1], "Lahan_Terbangun":[2], "Lainnya":"rest"}
for gname, members in GROUPS.items():
    if members == "rest":
        idx = [i for i in range(suits.shape[0]) if i not in used]
    else:
        idx = [cls_list.index(m) for m in members if m in cls_list]
    stacks.append(np.nansum(suits[idx,...], axis=0).astype(np.float32) if len(idx) else np.zeros((H,W), np.float32))
    used.update(idx)
SUIT_3 = np.stack(stacks, axis=0).astype(np.float32)
ss3 = SUIT_3.sum(axis=0, keepdims=True); ss3[ss3==0]=1; SUIT_3 /= ss3
suit_BU = SUIT_3[1]

# -------------------- Constraint Ske_B4B5 → eligibility --------------------
B4B5_raw, _ = read_raster(os.path.join(SAWAH_DIR, "Ske_B4B5.tif"))
IN_B4B5 = (np.isfinite(B4B5_raw) & (B4B5_raw > 0))

sawah_in_b4b5 = (PL_24==1) & IN_B4B5
n_sawah_b4b5  = int(sawah_in_b4b5.sum())
quota10       = int(np.floor(0.10 * n_sawah_b4b5)) if n_sawah_b4b5>0 else 0

nb2 = neighborhood_fraction(PL_24, 2, size=7)
cand_score = 0.8*suit_BU + 0.2*nb2

ALLOW_SAWAH_B4B5 = np.zeros_like(PL_24, dtype=bool)
if quota10 > 0:
    idx = np.flatnonzero(sawah_in_b4b5.ravel())
    pick = idx if idx.size <= quota10 else idx[np.argsort(-cand_score.ravel()[idx])[:quota10]]
    ALLOW_SAWAH_B4B5.ravel()[pick] = True

# Eligibility: outside B4B5 = free; inside B4B5 = only top 10% allowed
ELIGIBILITY = (~IN_B4B5) | ALLOW_SAWAH_B4B5
valid = (PL_24 != NODATA)
print(f"10% quota of rice fields in B4B5 allowed: {quota10} pixels")
print("Eligible pixel proportion:", float((ELIGIBILITY & valid).mean()))

# -------------------- Pressure (no Markov) --------------------
n2_2024 = neighborhood_fraction(PL_24, 2, NEIGH_SIZE)
pressure_proj = (1.0 - NEIGH_WEIGHT)*suit_BU + NEIGH_WEIGHT*n2_2024

# --- Edge softener: reduce high spikes of 1→2 near dense BU ---
edge_softener = 1.0 - 0.5 * np.clip(n2_2024, 0.0, 1.0)  # up to 50% reduction near BU edges

# -------------------- Endogenous calibration (non-BU→BU) --------------------
valid16 = (PL_16 != NODATA); valid24 = (PL_24 != NODATA)
nonBU16 = valid16 & ((PL_16==1) | (PL_16==3))
toBU_glob = nonBU16 & valid24 & (PL_24==2)
denom_glob = int(nonBU16.sum()); num_glob = int(toBU_glob.sum())
r_global = (num_glob / denom_glob) if denom_glob > 0 else 0.0
print(f"[Calibration] r_global (2016→2024, non-BU→BU) = {r_global:.8f} | samples={denom_glob}")

cand_proj = ELIGIBILITY & valid & ((PL_24==1) | (PL_24==3))
N_cand = int(cand_proj.sum())

E = r_global * N_cand
if E < 1 and N_cand > 0: E = max(1.0, 0.001 * N_cand)
E *= POLICY_INTENSITY

sum_press = float(np.nansum(pressure_proj[cand_proj])) if N_cand > 0 else 1.0
alpha0 = (E / sum_press) if sum_press > 0 else 0.0

p_map = np.zeros_like(pressure_proj, dtype=np.float32)
p_map[cand_proj] = alpha0 * pressure_proj[cand_proj]

# -------------------- Source weighting + protection & soft caps --------------------
mask1 = cand_proj & (PL_24==1)
mask3 = cand_proj & (PL_24==3)

p_map[mask1] *= W1 * edge_softener[mask1]   # edge softener for rice fields
p_map[mask3] *= W3

mask1_B4B5 = mask1 & IN_B4B5
p_map[mask1_B4B5] *= PROTECT_B4B5_FACTOR

# Apply global & local caps
p_map[mask1]      = np.minimum(p_map[mask1],      CAP_P_SAWAH)
p_map[mask1_B4B5] = np.minimum(p_map[mask1_B4B5], CAP_P_B4B5)

# -------------------- Source sharing (proportional pressure; still non-quota) --------------------
if USE_SOURCE_SHARE:
    cand1 = mask1
    cand3 = mask3
    E1 = E * E1_SHARE
    E3 = E * (1.0 - E1_SHARE)

    p1 = np.zeros_like(p_map, dtype=np.float32); p1[cand1] = p_map[cand1]
    p3 = np.zeros_like(p_map, dtype=np.float32); p3[cand3] = p_map[cand3]

    sum_p1 = float(np.nansum(p1[cand1])); sum_p3 = float(np.nansum(p3[cand3]))
    p_map[:] = 0.0
    if sum_p1 > 0: p_map[cand1] += p1[cand1] * (E1 / sum_p1)
    if sum_p3 > 0: p_map[cand3] += p3[cand3] * (E3 / sum_p3)

sum_p = float(np.nansum(p_map[cand_proj]))
if sum_p > 0: p_map[cand_proj] *= (E / sum_p)

p_map = np.clip(p_map, 0.0, 0.9)
print(f"[Prob-map] N_cand={N_cand} | E≈{E:.2f} | alpha0={alpha0:.6e} | Σp(after weights)≈{float(p_map[cand_proj].sum()):.2f}")

# -------------------- Probabilistic CA (BU absorbing; only 1/3→2) --------------------
rng = np.random.default_rng(RAND_SEED)

def step_probabilistic_CA(current_map, p_map, eligibility_mask, nodata=255):
    out = current_map.copy()
    valid = (current_map != nodata)
    out[current_map==2] = 2   # BU absorbing
    cand_to_BU = valid & eligibility_mask & ((current_map==1) | (current_map==3))
    if np.any(cand_to_BU):
        toss = rng.random(current_map.shape, dtype=np.float32)
        to_BU = cand_to_BU & (toss < p_map)
        out[to_BU] = 2
    out[~valid] = nodata
    return out

cur = PL_24.copy()
for _ in range(QC_ITERS):
    cur = step_probabilistic_CA(cur, p_map, ELIGIBILITY, nodata=NODATA)
pred_2032 = cur

# -------------------- Save outputs & summary --------------------
out_tif = os.path.join(CHGDIR, "Prediksi_2032_CA_prob_SkeB4B5_ULTRAprotect_noMarkov.tif")
save_with_colormap(pred_2032, PROFILE, out_tif)

ha_pix = ha_per_pixel(PROFILE)
luas24 = area_by_class(PL_24, ha_pix)
luas32 = area_by_class(pred_2032, ha_pix)

summary = pd.DataFrame([
    ["Sawah",     luas24[1], luas32[1], luas32[1]-luas24[1]],
    ["Terbangun", luas24[2], luas32[2], luas32[2]-luas24[2]],
    ["Lainnya",   luas24[3], luas32[3], luas32[3]-luas24[3]],
], columns=["Class","2024 (ha)","2032 — Ske_B4B5 (ha)","Δ(ha)"])
csv_out = os.path.join(REP_DIR, "area_summary_2032_SkeB4B5_ULTRAprotect_noMarkov.csv")
summary.to_csv(csv_out, index=False)

print("\n=== PROJECTION 2032 — Ske_B4B5 (Prob.; no Markov; BU absorbing; 10% adaptive; ULTRA-protected rice fields) ===")
print(summary.round(2))
print("✅ Saved:", out_tif)
print("✅ Saved:", csv_out)

# -------------------- Visualization --------------------
def show_classmap(arr, title, nodata=255):
    labels = ['Sawah (1)','Terbangun (2)','Lainnya (3)']
    cmap = ListedColormap(['#2e8b57','#dc143c','#1e90ff'])
    m = np.ma.masked_where(arr==nodata, arr)
    plt.imshow(m, cmap=cmap, vmin=1, vmax=3); plt.title(title); plt.axis('off')
    patches = [mpatches.Patch(color=cmap(i), label=labels[i]) for i in range(3)]
    return patches

plt.figure(figsize=(12,6), dpi=220)
plt.subplot(1,2,1); _ = show_classmap(PL_24, "OBSERVATION 2024")
plt.subplot(1,2,2); legend_handles = show_classmap(pred_2032, "PREDICTION 2032 — Ske_B4B5 (ULTRA protect)")
plt.legend(handles=legend_handles, loc='lower center', bbox_to_anchor=(0.5, -0.05), ncol=3, frameon=False)
plt.tight_layout(); plt.show()


# **UPI Scenario**

In [ ]:
# ============================================================
# UPI Projection — Probabilistic CA (Prioritize Sawah; Policy Intensity; no Markov)
# ============================================================
# Endogenous changes derived from historical rate → projected to eligible areas (Ske_UPI)
# through combined spatial pressures (road proximity, ZNT, neighborhood).
# Rice fields (class 1) are prioritized for protection, but other classes (3) may still convert.
# No manual target, fraction, or threshold is used.
# The total magnitude of change is controlled by POLICY_INTENSITY.
# ============================================================

# -------------------- Bootstrap: dependencies & mount --------------------
import sys, os
def _maybe_install_and_restart():
    try:
        import rasterio  # noqa
        import numpy as np  # noqa
        return
    except Exception as e:
        print("[Setup] Installing pinned versions for compatibility...", e)
        os.system(
            "pip -q install --upgrade --force-reinstall --no-cache-dir "
            " \"numpy==1.26.4\" \"rasterio==1.3.9\" "
            " \"scipy==1.11.4\" \"pandas==2.1.4\" \"matplotlib==3.8.2\" "
            " \"scikit-learn==1.3.2\" \"shapely==2.0.1\""
        )
        os.kill(os.getpid(), 9)
_maybe_install_and_restart()

try:
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive', force_remount=True)
except Exception:
    print("[Info] Not on Colab or Drive mount failed.")

# -------------------- Imports --------------------
import numpy as np
import rasterio
import pandas as pd
from scipy.ndimage import uniform_filter
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

# -------------------- Paths --------------------
SAWAH_DIR = "/content/drive/MyDrive/Sawah2"    # PL_2016.tif, PL_2024.tif, SUIT_2016_all.npy, Ske_UPI.tif
CHGDIR    = "/content/drive/MyDrive/Land Use/Change_Analysis"
REP_DIR   = os.path.join(CHGDIR, "reports")
os.makedirs(CHGDIR, exist_ok=True); os.makedirs(REP_DIR, exist_ok=True)

# -------------------- Constants & CA Settings --------------------
NODATA         = 255
QC_ITERS       = 4           # number of CA iterations for spatial smoothing (not for quantity tuning)
NEIGH_WEIGHT   = 0.40
NEIGH_SIZE     = 9
RAND_SEED      = 42          # reproducible results

# -------------------- Policy Preferences --------------------
W1 = 4.0   # multiplier to strengthen transition probability for sawah (class 1)
W3 = 0.6   # mild reduction for other classes (class 3) — still allowed to change
POLICY_INTENSITY = 3.0  # >1 increases expansion pressure (2.0–4.0 typical). Not a manual target.

# -------------------- Utility Functions --------------------
def read_raster(path):
    if not os.path.exists(path): raise FileNotFoundError(f"File not found: {path}")
    with rasterio.open(path) as src: return src.read(1), src.profile

def save_with_colormap(arr, profile, path, nodata=255):
    prof = profile.copy()
    prof.update(dtype=rasterio.uint8, nodata=nodata, count=1, compress="lzw",
                height=arr.shape[0], width=arr.shape[1])
    with rasterio.open(path, 'w', **prof) as dst:
        dst.write(arr.astype(np.uint8), 1)
        dst.write_colormap(1, {
            1:(46,139,87,255),   # Sawah
            2:(220,20,60,255),   # Terbangun
            3:(30,144,255,255),  # Lainnya
            255:(0,0,0,0)
        })
    print("✅ Saved:", path)

def ha_per_pixel(profile):
    t = profile['transform']; return abs(t.a)*abs(t.e)/10_000.0

def area_by_class(arr, ha_pix):
    return {c: float((arr==c).sum())*ha_pix for c in (1,2,3)}

def neighborhood_fraction(label_map, cls, size=5):
    return uniform_filter((label_map==cls).astype(np.float32), size=size, mode='nearest')

# -------------------- Load Maps & Suitability --------------------
PL_16, _       = read_raster(os.path.join(SAWAH_DIR, "PL_2016.tif"))
PL_24, PROFILE = read_raster(os.path.join(SAWAH_DIR, "PL_2024.tif"))
H, W = PL_24.shape

SUIT_ALL_PATH     = os.path.join(SAWAH_DIR, "SUIT_2016_all.npy")
SUIT_CLASSES_PATH = os.path.join(SAWAH_DIR, "SUIT_2016_all_classes.npy")
suits = np.load(SUIT_ALL_PATH)
if suits.ndim != 3 or suits.shape[1:] != (H,W):
    raise ValueError(f"SUIT_2016_all.npy dimensions mismatch: {suits.shape}; expected (C,{H},{W})")
if os.path.exists(SUIT_CLASSES_PATH): suit_classes = np.load(SUIT_CLASSES_PATH)
else: suit_classes = np.arange(suits.shape[0])

# Normalize channels
suits = np.nan_to_num(suits.astype(np.float32), nan=0.0, posinf=0.0, neginf=0.0)
ss = suits.sum(axis=0, keepdims=True); ss[ss==0]=1; suits = suits/ss

# Aggregate into 3 classes: [Sawah(1), Built-up(2), Others(3)]
GROUPS = {"Sawah":[1], "Lahan_Terbangun":[2], "Lainnya":"rest"}
cls_list = list(suit_classes.tolist()); stacks, names, used = [], [], set()
for gname, members in GROUPS.items():
    if members == "rest":
        idx = [i for i in range(suits.shape[0]) if i not in used]
    else:
        idx = [cls_list.index(m) for m in members if m in cls_list]
    stacks.append(np.nansum(suits[idx,...], axis=0).astype(np.float32) if len(idx) else np.zeros((H,W), np.float32))
    used.update(idx); names.append(gname)
SUIT_3 = np.stack(stacks, axis=0).astype(np.float32)
ss3 = SUIT_3.sum(axis=0, keepdims=True); ss3[ss3==0]=1; SUIT_3 = SUIT_3/ss3
print("Aggregated channels:", names)

# -------------------- UPI Constraint --------------------
C_raw, _ = read_raster(os.path.join(SAWAH_DIR, "Ske_UPI.tif"))
if C_raw.shape != (H,W): raise ValueError(f"Ske_UPI shape {C_raw.shape} ≠ map shape {PL_24.shape}. Please align grids first.")
eligible = np.zeros_like(C_raw, dtype=bool)
eligible[np.isfinite(C_raw) & (C_raw > 0)] = True
valid = (PL_24 != NODATA)
print("UPI Eligibility — proportion of eligible pixels:", float((eligible & valid).mean()))

# -------------------- Pressure (consistent with CA; no Markov) --------------------
n2_2016 = neighborhood_fraction(PL_16, 2, NEIGH_SIZE)
n2_2024 = neighborhood_fraction(PL_24, 2, NEIGH_SIZE)
suit_BU  = SUIT_3[1]
pressure_hist = (1.0 - NEIGH_WEIGHT)*suit_BU + NEIGH_WEIGHT*n2_2016
pressure_proj = (1.0 - NEIGH_WEIGHT)*suit_BU + NEIGH_WEIGHT*n2_2024

# -------------------- Endogenous change rate calibration --------------------
# A) Eligible-only rate (strictest under constraint)
mask_hist_cand = eligible & (PL_16 != NODATA) & ((PL_16==1) | (PL_16==3))
changed_to_BU  = mask_hist_cand & (PL_24==2)
denom_elig = int(mask_hist_cand.sum())
num_elig   = int(changed_to_BU.sum())
p_base_elig = (num_elig / denom_elig) if denom_elig > 0 else 0.0
print(f"[Calibration] p_base_elig (2016→2024, eligible non-BU→BU) = {p_base_elig:.8f} | samples={denom_elig}")

# B) Global fallback rate
valid16 = (PL_16 != NODATA); valid24 = (PL_24 != NODATA)
nonBU16 = valid16 & ((PL_16==1) | (PL_16==3))
toBU_glob = nonBU16 & valid24 & (PL_24==2)
denom_glob = int(nonBU16.sum())
num_glob   = int(toBU_glob.sum())
r_global = (num_glob / denom_glob) if denom_glob > 0 else 0.0
print(f"[Calibration] r_global (2016→2024, non-BU→BU, all valid) = {r_global:.8f} | samples={denom_glob}")

# C) Projection candidates: eligible & non-BU in 2024 (both 1 and 3)
cand_proj = eligible & valid & ((PL_24==1) | (PL_24==3))
N_cand = int(cand_proj.sum())

# D) Expected total transitions (no manual target), amplified by policy
if p_base_elig > 0:
    E = p_base_elig * N_cand
else:
    E = r_global * N_cand
    if E < 1 and N_cand > 0:
        E = max(1.0, 0.001 * N_cand)
E *= POLICY_INTENSITY  # strengthens expansion pressure

# E) Normalize pressure to form baseline probability so Σ p ≈ E
sum_press = float(np.nansum(pressure_proj[cand_proj])) if N_cand > 0 else 1.0
alpha0 = (E / sum_press) if sum_press > 0 else 0.0

p_map = np.zeros_like(pressure_proj, dtype=np.float32)
p_map[cand_proj] = alpha0 * pressure_proj[cand_proj]

# F) Apply source preference: rice fields more dominant, others still allowed
mask1 = cand_proj & (PL_24==1)
mask3 = cand_proj & (PL_24==3)
p_map[mask1] *= W1
p_map[mask3] *= W3

# Re-normalize so Σp(after weights) ≈ E (still no fixed target)
sum_p = float(np.nansum(p_map[cand_proj]))
if sum_p > 0:
    scale = E / sum_p
    p_map[cand_proj] *= scale

# Probability bounds for stochastic stability
p_map = np.clip(p_map, 0.0, 0.9)
print(f"[Prob-map] N_cand={N_cand} | E_raw≈{E:.2f} | alpha0={alpha0:.6e} | Σp(after weights)≈{float(p_map[cand_proj].sum()):.2f}")

# -------------------- Probabilistic CA (no quantity target) --------------------
rng = np.random.default_rng(RAND_SEED)

def step_probabilistic_CA(current_map, p_map, allowed_change_mask, nodata=255):
    """One CA step: eligible non-BU (1 & 3) may transition to BU according to p_map."""
    out = current_map.copy()
    valid = (current_map != nodata)
    cand  = valid & allowed_change_mask & ((current_map==1) | (current_map==3))
    if not np.any(cand): return out
    toss = rng.random(current_map.shape, dtype=np.float32)
    to_BU = cand & (toss < p_map)
    out[to_BU] = 2
    return out

cur = PL_24.copy()
for _ in range(QC_ITERS):
    cur = step_probabilistic_CA(cur, p_map, eligible, nodata=NODATA)
pred_2032 = cur

# -------------------- Save & Summary --------------------
out_tif = os.path.join(CHGDIR, "Prediksi_2032_UPI_CA_prob_priorSawah_POLICY_noquota_nomarkov.tif")
save_with_colormap(pred_2032, PROFILE, out_tif)

ha_pix = ha_per_pixel(PROFILE)
luas24 = area_by_class(PL_24, ha_pix)
luas32 = area_by_class(pred_2032, ha_pix)

summary = pd.DataFrame([
    ["Sawah",     luas24[1], luas32[1], luas32[1]-luas24[1]],
    ["Terbangun", luas24[2], luas32[2], luas32[2]-luas24[2]],
    ["Lainnya",   luas24[3], luas32[3], luas32[3]-luas24[3]],
], columns=["Class","2024 (ha)","2032 UPI (ha)","Δ(ha)"])

csv_out = os.path.join(REP_DIR, "area_summary_2032_UPI_CA_prob_priorSawah_POLICY_noquota_nomarkov.csv")
summary.to_csv(csv_out, index=False)

print("\n=== PROJECTION 2032 — UPI (Probabilistic CA; prioritize rice fields; policy-intensity; no Markov) ===")
print(f"[Info] Δ Built-up (ha) ≈ {(luas32[2]-luas24[2]):.2f} | Δ Sawah (ha) ≈ {(luas32[1]-luas24[1]):.2f}")
print(summary.round(2))
print("✅ Saved:", out_tif)
print("✅ Saved:", csv_out)

# -------------------- Visualization --------------------
def show_classmap(arr, title, nodata=255):
    labels = ['Sawah (1)','Terbangun (2)','Lainnya (3)']
    cmap = ListedColormap(['#2e8b57','#dc143c','#1e90ff'])
    m = np.ma.masked_where(arr==nodata, arr)
    plt.imshow(m, cmap=cmap, vmin=1, vmax=3); plt.title(title); plt.axis('off')
    patches = [mpatches.Patch(color=cmap(i), label=labels[i]) for i in range(3)]
    return patches

plt.figure(figsize=(12,6), dpi=600)
plt.subplot(1,2,2); legend_handles = show_classmap(pred_2032, "UPI Scenario")
plt.legend(handles=legend_handles, loc='lower center', bbox_to_anchor=(0.5, -0.05), ncol=3, frameon=False)
plt.tight_layout(); plt.show()


# **HYBRID Scenario**

In [ ]:
# ============================================================
# HYBRID Projection — Probabilistic CA (UPI ∪ B4B5-10%) — NO MARKOV
# ============================================================
# - No transition matrix (no T/T²)
# - HYBRID eligibility = Ske_UPI ∪ (10% of B4B5 rice fields adaptively selected by pressure)
# - Built-up (BU) class is absorbing; only 1/3 → 2 transitions allowed (1→3 implicitly blocked)
# - Pressure: built-up suitability + built-up neighborhood density
# - Non-quota mode: E = r_global(2016→2024, non-BU→BU) × POLICY_INTENSITY
# - Farmland-loss suppression: low source-share, B4B5 protection (penalty + soft cap), edge softener
# ============================================================

# -------------------- Bootstrap: dependencies & mount --------------------
import sys, os
def _maybe_install_and_restart():
    try:
        import rasterio  # noqa
        import numpy as np  # noqa
        return
    except Exception as e:
        print("[Setup] Installing pinned versions...", e)
        os.system(
            "pip -q install --upgrade --force-reinstall --no-cache-dir "
            " \"numpy==1.26.4\" \"rasterio==1.3.9\" "
            " \"scipy==1.11.4\" \"pandas==2.1.4\" \"matplotlib==3.8.2\" "
            " \"scikit-learn==1.3.2\" \"shapely==2.0.1\""
        )
        os.kill(os.getpid(), 9)
_maybe_install_and_restart()

try:
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive', force_remount=True)
except Exception:
    print("[Info] Not on Colab or Drive mount failed. Proceeding.")

# -------------------- Imports --------------------
import numpy as np, rasterio, pandas as pd
from scipy.ndimage import uniform_filter
import matplotlib.pyplot as plt, matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

# -------------------- Paths --------------------
SAWAH_DIR = "/content/drive/MyDrive/Sawah2"    # PL_2016.tif, PL_2024.tif, SUIT_2016_all.npy, Ske_UPI.tif, Ske_B4B5.tif
CHGDIR    = "/content/drive/MyDrive/Land Use/Change_Analysis"
REP_DIR   = os.path.join(CHGDIR, "reports")
os.makedirs(CHGDIR, exist_ok=True); os.makedirs(REP_DIR, exist_ok=True)

# -------------------- Constants & Policy Settings --------------------
NODATA       = 255
NEIGH_WEIGHT = 0.40
NEIGH_SIZE   = 9
QC_ITERS     = 3           # reduce iterations to limit repeated 1→2 conversions
RAND_SEED    = 42
np.random.seed(RAND_SEED)

# ===== Policy and Preferences (protective toward rice fields; still non-quota) =====
POLICY_INTENSITY = 2.2     # total expansion scaling factor (2.0–2.6 typical)
# Source weights (enhance 3→2, suppress 1→2)
W1_UPI  = 3.0              # rice fields eligible via UPI
W1_B4B5 = 0.6              # rice fields eligible only via B4B5 (weakened)
W3_ALL  = 0.8              # class 3 allowed (increase to divert more transitions)
# General protection for rice fields within B4B5 (beyond 10% allowance)
PROTECT_B4B5_FACTOR = 0.35
# Soft caps for 1→2 transitions (optional but effective)
USE_SOFT_CAPS  = True
CAP_P_SAWAH    = 0.04      # global cap for 1→2 probability (per iteration)
CAP_P_B4B5     = 0.01      # stricter cap for 1→2 within B4B5
# Source sharing: allocate expected E between rice-field and other sources (pressure-proportional)
USE_SOURCE_SHARE = True
E1_SHARE = 0.40            # share of E from rice-field sources (lower → smaller Δ sawah)
# Edge softener: reduce 1→2 probabilities near dense BU edges
EDGE_SOFTENER_STRENGTH = 0.5   # 0..1; 0.5 = 50% reduction near dense BU zones

# -------------------- Utility Functions --------------------
def read_raster(path):
    if not os.path.exists(path): raise FileNotFoundError(f"File not found: {path}")
    with rasterio.open(path) as src: return src.read(1), src.profile

def save_with_colormap(arr, profile, path, nodata=255):
    prof = profile.copy()
    prof.update(dtype=rasterio.uint8, nodata=nodata, count=1, compress="lzw",
                height=arr.shape[0], width=arr.shape[1])
    with rasterio.open(path, 'w', **prof) as dst:
        dst.write(arr.astype(np.uint8), 1)
        dst.write_colormap(1, {1:(46,139,87,255), 2:(220,20,60,255), 3:(30,144,255,255), 255:(0,0,0,0)})
    print("✅ Saved:", path)

def ha_per_pixel(profile):
    t = profile['transform']; return abs(t.a)*abs(t.e)/10_000.0

def area_by_class(arr, ha_pix):
    return {c: float((arr==c).sum())*ha_pix for c in (1,2,3)}

def neighborhood_fraction(label_map, cls, size=5):
    return uniform_filter((label_map==cls).astype(np.float32), size=size, mode='nearest')

# -------------------- Load Maps --------------------
PL_16,_       = read_raster(os.path.join(SAWAH_DIR,"PL_2016.tif"))
PL_24, PROF24 = read_raster(os.path.join(SAWAH_DIR,"PL_2024.tif"))
H, W = PL_24.shape

SUIT_ALL = np.load(os.path.join(SAWAH_DIR,"SUIT_2016_all.npy")).astype(np.float32)
if SUIT_ALL.ndim != 3 or SUIT_ALL.shape[1:] != (H,W):
    raise ValueError(f"SUIT_2016_all.npy shape {SUIT_ALL.shape} ≠ (C,{H},{W})")

SUIT_ALL = np.nan_to_num(SUIT_ALL, nan=0.0, posinf=0.0, neginf=0.0)
ss = SUIT_ALL.sum(axis=0, keepdims=True); ss[ss==0]=1; SUIT_ALL /= ss

classes_path = os.path.join(SAWAH_DIR,"SUIT_2016_all_classes.npy")
if os.path.exists(classes_path):
    suit_classes = np.load(classes_path)
else:
    suit_classes = np.arange(SUIT_ALL.shape[0])
cls_list = list(suit_classes.tolist())

def agg_to_3(SUIT_ALL, cls_list):
    stacks, used = [], set()
    for members in ([1],[2],"rest"):
        if members == "rest":
            idx = [i for i in range(SUIT_ALL.shape[0]) if i not in used]
        else:
            idx = [cls_list.index(m) for m in members if m in cls_list]
        stacks.append(np.nansum(SUIT_ALL[idx,...], axis=0).astype(np.float32) if len(idx) else np.zeros((H,W), np.float32))
        used.update(idx)
    SU3 = np.stack(stacks, axis=0).astype(np.float32)
    s = SU3.sum(axis=0, keepdims=True); s[s==0]=1
    return SU3/s

SUIT_3 = agg_to_3(SUIT_ALL, cls_list)
suit_BU = SUIT_3[1]

# -------------------- HYBRID Eligibility: UPI ∪ 10% of B4B5 Rice Fields --------------------
ELIG_UPI_raw, _ = read_raster(os.path.join(SAWAH_DIR,"Ske_UPI.tif"))
ELIG_UPI = (np.isfinite(ELIG_UPI_raw) & (ELIG_UPI_raw==1))

B4B5_raw, _ = read_raster(os.path.join(SAWAH_DIR,"Ske_B4B5.tif"))
IN_B4B5 = (np.isfinite(B4B5_raw) & (B4B5_raw>0))

sawah_in_b4b5 = (PL_24==1) & IN_B4B5
n_prot = int(sawah_in_b4b5.sum()); quota = int(np.floor(0.10*n_prot)) if n_prot>0 else 0

# Select 10% of B4B5 rice fields adaptively: suit_BU + BU neighborhood
nb2 = neighborhood_fraction(PL_24, 2, size=7)
cand_score = 0.8*suit_BU + 0.2*nb2
ALLOW_SAWAH_B4B5 = np.zeros_like(PL_24, dtype=bool)
if quota>0:
    idx = np.flatnonzero(sawah_in_b4b5.ravel())
    pick = idx if idx.size<=quota else idx[np.argsort(-cand_score.ravel()[idx])[:quota]]
    ALLOW_SAWAH_B4B5.ravel()[pick] = True

allowed_change = ELIG_UPI | ALLOW_SAWAH_B4B5
valid = (PL_24 != NODATA)
print("HYBRID Eligibility — proportion of eligible pixels:", float((allowed_change & valid).mean()))

# -------------------- Pressure (no Markov) --------------------
n2_2024 = neighborhood_fraction(PL_24, 2, NEIGH_SIZE)
pressure_proj = (1.0 - NEIGH_WEIGHT)*suit_BU + NEIGH_WEIGHT*n2_2024

# Edge softener for rice fields
edge_softener = 1.0 - EDGE_SOFTENER_STRENGTH * np.clip(n2_2024, 0.0, 1.0)

# -------------------- Endogenous Calibration (non-BU→BU, 2016→2024) --------------------
valid16 = (PL_16 != NODATA); valid24 = (PL_24 != NODATA)
nonBU16 = valid16 & ((PL_16==1) | (PL_16==3))
toBU_glob = nonBU16 & valid24 & (PL_24==2)
denom_glob = int(nonBU16.sum()); num_glob = int(toBU_glob.sum())
r_global = (num_glob / denom_glob) if denom_glob>0 else 0.0
print(f"[Calibration] r_global (2016→2024, non-BU→BU) = {r_global:.8f} | samples={denom_glob}")

cand_proj = allowed_change & valid & ((PL_24==1) | (PL_24==3))
N_cand = int(cand_proj.sum())

E = r_global * N_cand
if E < 1 and N_cand > 0: E = max(1.0, 0.001 * N_cand)
E *= POLICY_INTENSITY

sum_press = float(np.nansum(pressure_proj[cand_proj])) if N_cand>0 else 1.0
alpha0 = (E / sum_press) if sum_press>0 else 0.0

p_map = np.zeros_like(pressure_proj, dtype=np.float32)
p_map[cand_proj] = alpha0 * pressure_proj[cand_proj]

# -------------------- Source Weights + B4B5 Protection + Soft Caps --------------------
mask1_upi        = cand_proj & (PL_24==1) & ELIG_UPI
mask1_b4b5_allow = cand_proj & (PL_24==1) & (~ELIG_UPI) & ALLOW_SAWAH_B4B5
mask1_b4b5_all   = cand_proj & (PL_24==1) & IN_B4B5
mask3_any        = cand_proj & (PL_24==3)

p_map[mask1_upi]        *= W1_UPI * edge_softener[mask1_upi]
p_map[mask1_b4b5_allow] *= W1_B4B5 * edge_softener[mask1_b4b5_allow]
p_map[mask3_any]        *= W3_ALL

# Apply general penalty for all rice fields in B4B5 corridor (even beyond 10% allowance)
p_map[mask1_b4b5_all]   *= PROTECT_B4B5_FACTOR

# Apply soft caps for 1→2
if USE_SOFT_CAPS:
    mask1_all = cand_proj & (PL_24==1)
    p_map[mask1_all]      = np.minimum(p_map[mask1_all],      CAP_P_SAWAH)
    p_map[mask1_b4b5_all] = np.minimum(p_map[mask1_b4b5_all], CAP_P_B4B5)

# -------------------- Source Sharing (pressure-proportional; non-quota) --------------------
if USE_SOURCE_SHARE:
    cand1 = cand_proj & (PL_24==1)
    cand3 = cand_proj & (PL_24==3)
    E1 = E * E1_SHARE
    E3 = E * (1.0 - E1_SHARE)

    p1 = np.zeros_like(p_map, dtype=np.float32); p1[cand1] = p_map[cand1]
    p3 = np.zeros_like(p_map, dtype=np.float32); p3[cand3] = p_map[cand3]

    sum_p1 = float(np.nansum(p1[cand1])); sum_p3 = float(np.nansum(p3[cand3]))
    p_map[:] = 0.0
    if sum_p1 > 0: p_map[cand1] += p1[cand1] * (E1 / sum_p1)
    if sum_p3 > 0: p_map[cand3] += p3[cand3] * (E3 / sum_p3)

# Global re-normalization so Σp ≈ E
sum_p = float(np.nansum(p_map[cand_proj]))
if sum_p > 0:
    p_map[cand_proj] *= (E / sum_p)

p_map = np.clip(p_map, 0.0, 0.9)
print(f"[Prob-map] N_cand={N_cand} | E≈{E:.2f} | alpha0={alpha0:.6e} | Σp(after weights)≈{float(p_map[cand_proj].sum()):.2f}")

# -------------------- Probabilistic CA (BU absorbing; only 1/3→2) --------------------
rng = np.random.default_rng(RAND_SEED)
def step_probabilistic_CA(current_map, p_map, eligibility_mask, nodata=255):
    out = current_map.copy()
    valid = (current_map != nodata)
    out[current_map==2] = 2  # BU absorbing
    cand_to_BU = valid & eligibility_mask & ((current_map==1) | (current_map==3))
    if np.any(cand_to_BU):
        toss = rng.random(current_map.shape, dtype=np.float32)
        to_BU = cand_to_BU & (toss < p_map)
        out[to_BU] = 2
    out[~valid] = nodata
    return out

cur = PL_24.copy()
for _ in range(QC_ITERS):
    cur = step_probabilistic_CA(cur, p_map, allowed_change, nodata=NODATA)

pred_2032_hybrid = cur

# -------------------- Summary & Save --------------------
out_tif = os.path.join(CHGDIR, "Prediksi_2032_HYBRID_Protective_noMarkov.tif")
save_with_colormap(pred_2032_hybrid, PROF24, out_tif)

ha_pix = ha_per_pixel(PROF24)
luas24 = area_by_class(PL_24, ha_pix)
luas32 = area_by_class(pred_2032_hybrid, ha_pix)
df = pd.DataFrame([
    ["Sawah",     luas24[1], luas32[1], luas32[1]-luas24[1]],
    ["Terbangun", luas24[2], luas32[2], luas32[2]-luas24[2]],
    ["Lainnya",   luas24[3], luas32[3], luas32[3]-luas24[3]],
], columns=["Class","2024 (ha)","2032 HYBRID (ha)","Δ(ha)"])
print("\n=== PROJECTION 2032 — HYBRID (Probabilistic; protective; no Markov; BU absorbing) ===")
print(df.round(2))

csv_out = os.path.join(REP_DIR, "area_summary_2032_HYBRID_Protective_noMarkov.csv")
df.to_csv(csv_out, index=False)
print("✅ Saved:", out_tif)
print("✅ Saved:", csv_out)

# -------------------- Visualization --------------------
def show_classmap(arr, title, nodata=255):
    labels = ['Sawah (1)','Terbangun (2)','Lainnya (3)']
    cmap = ListedColormap(['#2e8b57','#dc143c','#1e90ff'])
    m = np.ma.masked_where(arr==nodata, arr)
    plt.imshow(m, cmap=cmap, vmin=1, vmax=3); plt.title(title); plt.axis('off')
    patches = [mpatches.Patch(color=cmap(i), label=labels[i]) for i in range(3)]
    return patches

plt.figure(figsize=(12,6), dpi=220)
plt.subplot(1,2,1); _ = show_classmap(PL_24, "OBSERVATION 2024")
plt.subplot(1,2,2); legend_handles = show_classmap(pred_2032_hybrid, "PROJECTION 2032 — HYBRID (Protective)")
plt.legend(handles=legend_handles, loc='lower center', bbox_to_anchor=(0.5, -0.05), ncol=3, frameon=False)
plt.tight_layout(); plt.show()
